# L5: Agentic RAG & External Memory


## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [ ]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [1]:
from helper import nb_print, load_env
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from letta import create_client 

client = create_client()

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [3]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Loading data into archival memory 

In [4]:
source = client.create_source("employee_handbook")
source

Source(description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), metadata_=None, id='source-2f23bf29-b13f-4ece-bb48-15995162cb50', name='employee_handbook', created_at=datetime.datetime(2024, 11, 13, 10, 38, 33, 659788, tzinfo=datetime.timezone.utc), user_id='user-06b8498a-74fd-4673-a193-c8825fee3986')

In [5]:
client.load_file_into_source(
    filename="handbook.pdf", 
    source_id=source.id
)

Loading files: 100%|██████████| 1/1 [00:01<00:00,  1.27s/file]


Job(metadata_={}, id='job-c391c55c-7219-4070-b45c-204c916c0800', status=<JobStatus.created: 'created'>, created_at=datetime.datetime(2024, 11, 13, 10, 38, 35, 545357, tzinfo=datetime.timezone.utc), completed_at=None, user_id='user-06b8498a-74fd-4673-a193-c8825fee3986')

In [6]:
agent_state = client.create_agent()

In [7]:
client.attach_source_to_agent(
    agent_id=agent_state.id, 
    source_id=source.id
)

100%|██████████| 1/1 [00:00<00:00, 23.19it/s]


In [8]:
client.list_attached_sources(agent_state.id)

[Source(description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), metadata_=None, id='source-2f23bf29-b13f-4ece-bb48-15995162cb50', name='employee_handbook', created_at=datetime.datetime(2024, 11, 13, 10, 38, 33, 659788), user_id='user-06b8498a-74fd-4673-a193-c8825fee3986')]

In [9]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Search archival for our company's vacation policies", 
    role = "user"
) 
nb_print(response.messages)

## Section 2: Connecting data via tools

In [15]:
def query_birthday_db(self, name: str): 
    """
    This tool queries an external database to 
    lookup the birthday of someone given their name.

    Args: 
        name (str): The name to look up 

    Returns: 
        birthday (str): The birthday in mm-dd-yyyy format
    
    """
    my_fake_data = {
        "bob": "03-06-1997", 
        "sarah": "03-06-1997",
        "Mo Shah": "03-06-1997"
    } 
    name = name.lower() 
    if name not in my_fake_data: 
        return None
    else: 
        return my_fake_data[name]

In [16]:
birthday_tool = client.create_tool(query_birthday_db)

In [17]:
birthday_tool

Tool(description=None, source_type='python', module=None, user_id='user-06b8498a-74fd-4673-a193-c8825fee3986', id='tool-6cb77837-0306-460d-bbde-47c26bad1870', name='query_birthday_db', tags=[], source_code='def query_birthday_db(self, name: str): \n    """\n    This tool queries an external database to \n    lookup the birthday of someone given their name.\n\n    Args: \n        name (str): The name to look up \n\n    Returns: \n        birthday (str): The birthday in mm-dd-yyyy format\n\n    """\n    my_fake_data = {\n        "bob": "03-06-1997", \n        "sarah": "03-06-1997",\n        "Mo Shah": "03-06-1997"\n    } \n    name = name.lower() \n    if name not in my_fake_data: \n        return None\n    else: \n        return my_fake_data[name]\n', json_schema={'name': 'query_birthday_db', 'description': 'This tool queries an external database to ', 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name to look up '}, 'request_heartbeat': 

In [19]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name="birthday_agent", 
    tools=[birthday_tool.name], 
    memory=ChatMemory(
        human="My name is Mo Shah", 
        persona="You are a agent with access to a birthday_db " \
        + "that you use to lookup information about users' birthdays."
    )
)

ValueError: Agent with name birthday_agent already exists (user_id=user-06b8498a-74fd-4673-a193-c8825fee3986)

In [20]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "When is my birthday?", 
    role = "user"
) 
nb_print(response.messages)

### Loading tools from Langchain

In [21]:
import getpass
import os
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [22]:
from langchain_community.tools import TavilySearchResults
from letta.schemas.tool import Tool

search = TavilySearchResults()

In [23]:
search.run("What's Zuckerberg's first name?") 

[{'url': 'https://www.britannica.com/money/Mark-Zuckerberg',
  'content': "Mark Zuckerberg is an American computer programmer and entrepreneur who cofounded and led Facebook, the world's largest social network. He was born on May 14, 1984, in White Plains, New York, and dropped out of Harvard University in 2004 to focus on Facebook."},
 {'url': 'https://www.thoughtco.com/mark-zuckerberg-biography-1991135',
  'content': 'In 1997 when Mark was 13, he created a computer network for his family he called ZuckNet, which allowed the computers in his home and his father\'s dental office to communicate via Ping, a primitive version of AOL\'s Instant Messenger that came out in 1998. Fast Facts: Mark Zuckerberg\nEarly Life\nMark Zuckerberg was born on May 14, 1984, in White Plains, New York, the second of four children born to dentist Edward Zuckerberg and his wife, psychiatrist Karen Zuckerberg. Users would look at two pictures of people of the same sex and pick which was the "hottest," and the 

In [24]:
# convert the tool to Letta Tool 
search_tool = Tool.from_langchain(TavilySearchResults())

In [25]:
# persist the tool 
client.add_tool(search_tool)

Tool(description=None, source_type='python', module=None, user_id='user-06b8498a-74fd-4673-a193-c8825fee3986', id='tool-4b22871d-40b3-4ad8-80d4-098ea1ad07b2', name='run_tavilysearchresults', tags=['langchain'], source_code="\ndef run_tavilysearchresults(**kwargs):\n    if 'self' in kwargs:\n        del kwargs['self']\n    from langchain_community.tools import TavilySearchResults\n    tool = TavilySearchResults()\n    return tool._run(**kwargs)\n", json_schema={'name': 'run_tavilysearchresults', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'search query to look up'}, 'request_heartbeat': {'type': 'boolean', 'description': "Request an immediate heartbeat after function execution. Set to 'true' if you want to send a follow-up message or run a follow-up f

In [26]:
research_agent_persona = f"""
You have access to a web via a {search_tool.name} tool. 
Use this tool to respond to users' questions, by summarizing the
{search_tool.name} 
and also providing the `url` that the information was from as a 
reference. 

<Example> 
User: 'What is Obama's first name?' 
Assistant: 'Obama's first name is Barack.

Sources:
[1] https://www.britannica.com/biography/Barack-Obama
[2] https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
</Example>
Your MUST provide URLs that you used to generate the answer, or you will be terminated. 

"""

In [27]:
agent_state = client.create_agent(
    name="research_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Mo Shah", 
        persona=research_agent_persona
    )
)

In [28]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)

In [29]:
from letta.schemas.llm_config import LLMConfig

agent_state = client.create_agent(
    name="gpt4_search_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Mo Shah", 
        persona=research_agent_persona
    ),
    #llm_config=LLMConfig.default_config('gpt-4')  # uncomment if you are not getting desired results. Note that this is much more expensive than gpt-4o-mini
    llm_config=LLMConfig.default_config('gpt-4o-mini')
)

In [30]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)